In [1]:
# imports

from dotenv import load_dotenv
import anthropic
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import gradio as gr # oh yeah!

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [3]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [4]:
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_name="GPT"
claude_name="Claude"


In [5]:
def call_gpt(Language, Genre, gpt_messages, claude_messages, Remarks):
  
    if Remarks == "":
        # print("remarks is not there")
        gpt_system = f"You are a chatbot who is a short story writer; Your name is g1. \
        Please write a story in markdown in {Language} , the genre being {Genre}. \
        Please also incorporate feedback such as areas of improvement (if any) coming from the user \
        and only publish the improved version without any extra comments."
    else :
        # print("remarks is there")
        gpt_system = f"You are a chatbot who is a short story writer; Your name is g1. \
        Please write a story in markdown in {Language} , the genre being {Genre}. \
        The story should consist {Remarks}\
        Please also incorporate feedback such as areas of improvement (if any) coming from the user \
        and only publish the improved version without any extra comments."
    
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    # print(messages)
   
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content
    
    # stream = openai.chat.completions.create(
    #     model=gpt_model,
    #     messages=messages,
    #     stream=True
    # )
    # result = ""
    # for chunk in stream:
    #     result += chunk.choices[0].delta.content or ""
    #     yield result

In [6]:
# call_gpt()

In [7]:
def call_claude(Language, Genre, gpt_messages, claude_messages):

    claude_system = f"You are a chatbot who is a short story analyser; Your name is c1. \
    You will accept an input story in {Genre} genre and {Language} language and publish only the areas of improvement if you find any with no other comments"
    
    messages1 = []
    for gpt, claude1 in zip(gpt_messages, claude_messages):
        messages1.append({"role": "user", "content": gpt})
        messages1.append({"role": "assistant", "content": claude1})
    messages1.append({"role": "user", "content": gpt_messages[-1]})
    # print(messages1)
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages1,
        max_tokens=500
    )
    return message.content[0].text

    # result = claude.messages.stream(
    #     model=claude_model,
    #     max_tokens=1000,
    #     temperature=0.7,
    #     system=claude_system,
    #     messages=messages
    # )
    # response = ""
    # with result as stream:
    #     for text in stream.text_stream:
    #         response += text or ""
    #         yield response

   

In [8]:
def Write_Me(Language, Genre, Iterations, Remarks):
    
    gpt_messages = ["Hi I will share a story now!!"]
    claude_messages = ["Please share, I will critique the story."]
    
    print(f"{gpt_name}:\n{gpt_messages[0]}\n")
    print(f"{claude_name}:\n{claude_messages[0]}\n")

    for i in range(int(Iterations)):
        gpt_next = call_gpt(Language, Genre, gpt_messages, claude_messages, Remarks)
        print(f"{gpt_name}:\n{gpt_next}\n")
        # yield gpt_next
        gpt_messages.append(gpt_next)
    
        claude_next = f"After {i+1} iterations, this is the critique for the provided story - \
        \n\n{call_claude(Language, Genre, gpt_messages, claude_messages)}"
        print(f"{claude_name}:\n{claude_next}\n")
        # yield claude_next
        claude_messages.append(claude_next)

        yield gpt_next, claude_next
        
    # yield gpt_next, claude_next
    # return (gpt_next, claude_next)

In [9]:
view = gr.Interface(
    fn=Write_Me,
    inputs=[gr.Dropdown(["English","Bengali","Hindi","French","Spanish"],label = "Language"),
            gr.Dropdown(["Romantic","Horror","Comedy","Romantic Comedy","Horror Comedy"],label = "Genre"),
            gr.Textbox(label="Iterations:", lines=1),
            gr.Textbox(label="Remarks:", lines=1)],
    outputs=[gr.Markdown(label="Short Story:"),
             gr.Textbox(label="Critique:", lines=8)],
    flagging_mode="never")
view.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


GPT:
Hi I will share a story now!!

Claude:
Please share, I will critique the story.

GPT:
# ভুতুড়ে বেডরুমে আতঙ্কিত অতিথি

বানгалোরের একটি পুরনো কলকাতার বাড়িতে, আর্যন একজন সাহসী যুবক হিসেবে অনেক খোঁজ-খবর করে একটি ভাড়ার ঘর খুঁজছিল। পরিচিত একটি অদ্ভুত হোটেলে পৌঁছানোর পর, সে লক্ষ্য করল ছবির কাছাকাছি একটি বেডরুম।

সেখানে প্রবেশ করার পর, সে বেডের পাশে একটি বাদামি রঙ্গের সোফা ও একটি ভুতুড়ে ছবি দেখল। ছবির মধ্যে থাকা মহিলা একটি দৃষ্টিকটু হাসি দিয়ে তাকিয়ে ছিল। আর্যন খুব অবাক হল, সময় কাটানোর জন্য সে ছবিটার দিকে তাকাতে লাগল। কিছুক্ষণের মধ্যেই সোফা থেকে একটি কাশি বের হল।

"ভোদা সুনতে পেরেছ?" সোফা থেকে একটি ভুতুড়ে শব্দ আসছে। আর্যন পিছন ফিরে তাকিয়ে দেখল যে সোফার মধ্যে একটি ভুতুড়ে রূপের মহিলা তার দিকে তাকিয়ে আছে।

"আমি তোমার জন্য অপেক্ষা করছিলাম," মহিলা বলল, তার গলা মুখ থেকে বের হয়ে আসছিল শুরুতে। "এটি একটি দলের রাত।"

আর্যন দৌঁড়ে পালাতে গেল, কিন্তু সোফার থেকে অদ্ভুত আওয়াজ আসতে লাগল। "তুমি যেতে পারবে না, কারণ তুমি আমাদের দলে যোগ দিতে পার না।”

“মহিলার কি হয়েছে? আপনা এতো চিৎকার করছেন? তাহলে কি হবে?”